# HW 3 - Question 7: Titanic Data Set 

## Question

Task 7:Learning SVMs on the Titanic dataset ((https://www.kaggle.com/c/titanic). Please report your fivefold
cross validation classification accuracies on Titanic training set, with respect to the linear, quadratic, and
RBF kernels. Which kernel is the best in your case?

### Load the libraries

In [34]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os
import random as rnd

## ML libraries

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn import svm
from sklearn import metrics

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# to make this notebook's output stable across runs
np.random.seed(42)

### Load the data 

In [17]:
trainpath='./Data/train.csv'

testpath='./Data/test.csv'


df_trn=pd.read_csv(trainpath)
df_trn.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
df_tst=pd.read_csv(testpath)

df_tst.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
combine = [df_trn,df_tst]

### Feature Identification

Numerical features:  'Age' 'SibSp' 'Parch' 'Fare' 
 
Catagorical:  'PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Ticket'  'Cabin' 'Embarked'



In [20]:
## Feature Selection Dropping ticket passenger and cabin

# not relevant to us

In [21]:
for d in combine:
    d=d.drop(['Ticket','PassengerId','Cabin'],1)

In [22]:
df_trn=df_trn.drop(['Ticket','PassengerId','Cabin'],1)
df_tst=df_tst.drop(['Ticket','PassengerId','Cabin'],1)
combine = [df_trn,df_tst]

In [23]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False) ## extract the first of the name
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [24]:
for dt in combine:
    dt["Embarked"]=dt["Embarked"].replace(to_replace=['C','Q','S'],value=[1,2,3])
    dt['Embarked'].fillna('S', inplace=True) ## using mode
    
    dt.Sex = dt['Sex'].replace(to_replace =["male","female"], value = [0,1])
    
    print(dt.head())
    

   Survived  Pclass                                               Name  Sex  \
0         0       3                            Braund, Mr. Owen Harris    0   
1         1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1   
2         1       3                             Heikkinen, Miss. Laina    1   
3         1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1   
4         0       3                           Allen, Mr. William Henry    0   

    Age  SibSp  Parch     Fare Embarked Title  
0  22.0      1      0   7.2500      3.0    Mr  
1  38.0      1      0  71.2833      1.0   Mrs  
2  26.0      0      0   7.9250      3.0  Miss  
3  35.0      1      0  53.1000      3.0   Mrs  
4  35.0      0      0   8.0500      3.0    Mr  
   Pclass                                          Name  Sex   Age  SibSp  \
0       3                              Kelly, Mr. James    0  34.5      0   
1       3              Wilkes, Mrs. James (Ellen Needs)    1  47.0      1   
2     

In [25]:

df_trn=df_trn.drop(['Name'],1)
df_tst=df_tst.drop(['Name'],1)
combine = [df_trn,df_tst]

In [26]:
df_trn

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,3.0,Mr
1,1,1,1,38.0,1,0,71.2833,1.0,Mrs
2,1,3,1,26.0,0,0,7.9250,3.0,Miss
3,1,1,1,35.0,1,0,53.1000,3.0,Mrs
4,0,3,0,35.0,0,0,8.0500,3.0,Mr
...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,3.0,Rare
887,1,1,1,19.0,0,0,30.0000,3.0,Miss
888,0,3,1,NaN,1,2,23.4500,3.0,Miss
889,1,1,0,26.0,0,0,30.0000,1.0,Mr


In [27]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [28]:
for df in combine:
    df['Age'].fillna(30.272590, inplace=True)
    df['Fare'].fillna(33, inplace=True)
    

## Model Inputs

In [29]:
X_train = df_trn.drop("Survived", axis=1)
Y_train = df_trn["Survived"]
X_test  = df_tst.copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [30]:
df_tst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    int64  
 7   Title     418 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 26.2 KB


In [31]:
X_train["Embarked"]=X_train["Embarked"].replace(to_replace=['C','Q','S'],value=[1,2,3])

In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    float64
 7   Title     891 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 55.8 KB


## SVM Modeling

In [35]:
svm_clf = svm.SVC(kernel='linear', C = 1.0)
svm_clf.fit(X_train,Y_train)

SVC(kernel='linear')

In [56]:
svm_clf = svm.SVC(kernel='linear', C = 1.0)
acc=[]

for train_index , test_index in kf.split(X_train):
    x , xt = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y , yt = Y_train[train_index],Y_train[test_index]
    svm_clf = svm_clf.fit(x,y)
    y_pred2 = svm_clf.predict(xt)
    acc.append(metrics.accuracy_score(yt, y_pred2))
    print(metrics.accuracy_score(yt, y_pred2))
    
print("Average Accuracy:", sum(acc)/5)

0.5714285714285714
0.7362637362637363
0.5824175824175825
0.5274725274725275
0.5604395604395604
Average Accuracy: 0.5956043956043956


In [57]:
svm_clf = svm.SVC(kernel='rbf')
acc=[]

for train_index , test_index in kf.split(X_train):
    x , xt = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y , yt = Y_train[train_index],Y_train[test_index]
    svm_clf = svm_clf.fit(x,y)
    y_pred2 = svm_clf.predict(xt)
    acc.append(metrics.accuracy_score(yt, y_pred2))
    print(metrics.accuracy_score(yt, y_pred2))
    
print("Average Accuracy:", sum(acc)/5)

0.5714285714285714
0.7802197802197802
0.5714285714285714
0.5274725274725275
0.5604395604395604
Average Accuracy: 0.6021978021978022


In [58]:
svm_clf = svm.SVC(kernel='poly', degree=3)
acc=[]

for train_index , test_index in kf.split(X_train):
    x , xt = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y , yt = Y_train[train_index],Y_train[test_index]
    svm_clf = svm_clf.fit(x,y)
    y_pred2 = svm_clf.predict(xt)
    acc.append(metrics.accuracy_score(yt, y_pred2))
    print(metrics.accuracy_score(yt, y_pred2))
    
print("Average Accuracy:", sum(acc)/5)

0.5714285714285714
0.7802197802197802
0.5714285714285714
0.5274725274725275
0.5604395604395604
Average Accuracy: 0.6021978021978022
